# Title

## Problem Description

## Imports and Utility Code

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import nltk
from collections import Counter
import syllapy
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
import ety
from tqdm import tqdm

import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, Input, Flatten
from keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LeakyReLU

tqdm.pandas()
nltk.download("wordnet")
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

c:\Users\focus\Develop\CU\DTSA-5511\final\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\focus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\focus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\focus\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Exploritory Data Analysis

In [2]:
df = pd.read_csv("CLEAR Corpus 6.01 - CLEAR Corpus 6.01.csv")
df = df[df["Kaggle split"] == "Train"]

In [3]:
df.info(max_cols=None)

<class 'pandas.core.frame.DataFrame'>
Index: 2834 entries, 0 to 4723
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  2834 non-null   int64  
 1   Last Changed                        84 non-null     float64
 2   Author                              2834 non-null   object 
 3   Title                               2834 non-null   object 
 4   Anthology                           1609 non-null   object 
 5   URL                                 2834 non-null   object 
 6   Source                              2834 non-null   object 
 7   Pub Year                            2829 non-null   float64
 8   Category                            2834 non-null   object 
 9   Location                            2834 non-null   object 
 10  License                             2834 non-null   object 
 11  MPAA
Max                            2834 non-nul

In [4]:
df.head(10)

,ID,Last Changed,Author,Title,Anthology,URL,Source,Pub Year,Category,Location,...,CAREC_M,CARES,CML2RI,firstPlace_pred,secondPlace_pred,thirdPlace_pred,fourthPlace_pred,fifthPlace_pred,sixthPlace_pred,Kaggle split
0,400,NaN,Carolyn Wells,Patty's Suitors,NaN,http://www.gutenberg.org/cache/epub/5631/pg563...,gutenberg,1914.0,Lit,mid,...,0.11952,0.457534,12.097815,-0.383831,-0.283604,-0.346879,-0.281620,-0.247767,-0.289945,Train
1,401,NaN,Carolyn Wells,Two Little Women on a Holiday,NaN,http://www.gutenberg.org/cache/epub/5893/pg589...,gutenberg,1917.0,Lit,mid,...,0.04921,0.462510,22.550179,-0.260307,-0.209960,-0.061565,-0.234231,-0.201347,-0.156156,Train
2,402,NaN,Carolyn Wells,Patty Blossom,NaN,http://www.gutenberg.org/cache/epub/20945/pg20...,gutenberg,1917.0,Lit,mid,...,0.09724,0.369259,18.125279,-0.615037,-0.530600,-0.527847,-0.550180,-0.565762,-0.538852,Train
4,404,NaN,Charles Kingsley,HOW THE ARGONAUTS WERE DRIVEN INTO THE UNKNOWN...,The Heroes\n or Greek Fairy Tales for my...,http://www.gutenberg.org/files/677/677-h/677-h...,gutenberg,1889.0,Lit,mid,...,0.08798,0.389226,3.195960,-1.335586,-1.321922,-1.163985,-1.122501,-1.185518,-1.271324,Train
5,405,NaN,Charles Madison Curry\n Erle Elsworth C...,The Three Little Bears,Children's Literature\n A Textbook of So...,http://www.gutenberg.org/files/25545/25545-h/2...,gutenberg,1920.0,Lit,mid,...,0.36885,0.301666,28.990105,0.341717,0.376123,0.353762,0.343373,0.361875,0.368739,Train
6,406,NaN,Clair W. Hayes,"The Boy Allies On the Firing Line\n Or, ...",NaN,http://www.gutenberg.org/files/12870/12870-h/1...,gutenberg,1915.0,Lit,mid,...,0.16523,0.419842,12.766583,-1.070515,-1.022543,-0.971631,-0.991998,-1.040679,-1.075716,Train
7,407,NaN,Clair W. Hayes,The Boy Allies in Great Peril,NaN,http://www.gutenberg.org/cache/epub/12682/pg12...,gutenberg,1916.0,Lit,mid,...,0.18656,0.484475,14.130141,-1.390635,-1.554880,-1.581937,-1.666938,-1.540613,-1.600628,Train
8,408,NaN,Clair W. Hayes,The Boy Allies At Verdun,NaN,http://www.gutenberg.org/cache/epub/13020/pg13...,gutenberg,1917.0,Lit,start,...,0.12905,0.430107,10.216473,-1.041028,-1.093127,-0.959339,-0.954064,-1.023613,-0.903814,Train
9,409,NaN,Claude A. Labelle,The Ranger Boys and the Border Smugglers,NaN,http://www.gutenberg.org/files/25514/25514-h/2...,gutenberg,1922.0,Lit,mid,...,0.07326,0.377270,16.497078,-0.273477,-0.281462,-0.281333,-0.280411,-0.311182,-0.310186,Train
10,411,NaN,Cornelius Mathews,THE CELESTIAL SISTERS,The Indian Fairy Book\n From the Origina...,http://www.gutenberg.org/files/22248/22248-h/2...,gutenberg,1869.0,Lit,mid,...,0.07593,0.454556,15.388692,-0.976894,-1.100460,-1.091416,-1.167673,-1.057143,-1.036163,Train


In [5]:
fig = px.histogram(df["BT Easiness"], nbins=30)
fig.update_layout(bargap=0.1)

fig.show()

In [6]:
correlation_matrix = df[
    [
        "BT Easiness",
        "Flesch-Reading-Ease",
        "Flesch-Kincaid-Grade-Level",
        "Automated Readability Index",
        "SMOG Readability",
        "New Dale-Chall Readability Formula",
        "CAREC",
        "CAREC_M",
        "CARES",
        "CML2RI",
    ]
].corr()

import plotly.graph_objects as go

fig = go.Figure(
    data=go.Heatmap(
        z=correlation_matrix.values,
        x=correlation_matrix.index,
        y=correlation_matrix.columns,
        colorscale="balance",  # Diverging color scale for positive and negative correlations
        colorbar=dict(title="Correlation"),
        text=correlation_matrix.values.round(2),
    )
)

fig.update_layout(
    title="Correlation Plot for Readability Indicies",
    xaxis_title="Columns",
    yaxis_title="Columns",
    height=600,
    width=600,
)

fig.show()

In [7]:
pos_mapping = {
    "CC": "Coordinating Conjunction",
    "CD": "Cardinal Digit",
    "DT": "Determiner",
    "EX": "Existential There",
    "FW": "Foreign Word",
    "IN": "Preposition or Subordinating Conjunction",
    "JJ": "Adjective",
    "JJR": "Adjective, Comparative",
    "JJS": "Adjective, Superlative",
    "LS": "List Item Marker",
    "MD": "Modal",
    "NN": "Noun, Singular or Mass",
    "NNS": "Noun, Plural",
    "NNP": "Proper Noun, Singular",
    "NNPS": "Proper Noun, Plural",
    "PDT": "Predeterminer",
    "POS": "Possessive Ending",
    "PRP": "Personal Pronoun",
    "PRP$": "Possessive Pronoun",
    "RB": "Adverb",
    "RBR": "Adverb, Comparative",
    "RBS": "Adverb, Superlative",
    "RP": "Particle",
    "TO": "to",
    "UH": "Interjection",
    "VB": "Verb, Base Form",
    "VBD": "Verb, Past Tense",
    "VBG": "Verb, Gerund or Present Participle",
    "VBN": "Verb, Past Participle",
    "VBP": "Verb, Non-3rd Person Singular Present",
    "VBZ": "Verb, 3rd Person Singular Present",
    "WDT": "Wh-determiner",
    "WP": "Wh-pronoun",
    "WP$": "Possessive Wh-pronoun",
    "WRB": "Wh-adverb",
}


def process_text(text):
    text = text.lower()

    word_origins = []
    word_pos = []
    syllable_counts = []
    sentence_lengths = []
    word_lengths = []

    sentences = sent_tokenize(text)

    for sentence in sentences:
        tokens = nltk.word_tokenize(sentence)
        pos_tags = nltk.pos_tag(tokens)
        sentence_lengths.append(len(pos_tags))
        for token, pos in pos_tags:
            origin = ety.origins(token)
            if origin:
                origin = origin[0].language.name
            else:
                origin = "unknown"
            word_origins.append(origin)
            full_pos_name = pos_mapping.get(pos, pos)
            word_pos.append(full_pos_name)
            syllables = syllapy.count(token)
            syllable_counts.append(syllables)
            word_lengths.append(len(token))

    processed_excerpt = text
    origin_counts = Counter(word_origins)
    pos_counts = Counter(word_pos)
    mean_syllable_count = np.mean(syllable_counts)
    num_sentences = len(sentences)
    mean_sentence_length = np.mean(sentence_lengths)
    num_words = np.sum(sentence_lengths)
    mean_word_length = np.mean(word_lengths)

    return (
        word_origins,
        origin_counts,
        word_pos,
        pos_counts,
        syllable_counts,
        mean_syllable_count,
        num_sentences,
        mean_sentence_length,
        mean_word_length,
        num_words,
        processed_excerpt,
    )

In [8]:
df[
    [
        "word_origins",
        "word_origin_counts",
        "pos",
        "pos_counts",
        "syllable_counts",
        "mean_syllable_count",
        "num_sentences",
        "mean_sentence_length",
        "mean_word_length",
        "num_words",
        "processed_excerpt",
    ]
] = df["Excerpt"].progress_apply(lambda x: pd.Series(process_text(x)))

100%|██████████| 2834/2834 [00:41<00:00, 67.72it/s]


In [9]:
origins_df = pd.DataFrame(df["BT Easiness"]).join(
    pd.DataFrame(df["word_origin_counts"].tolist()).fillna(0)
)

correlation_matrix = origins_df.corr()
bt_easiness_corr = correlation_matrix["BT Easiness"]
sorted_index = bt_easiness_corr.sort_values(ascending=False).index
bt_easiness_corr = bt_easiness_corr[sorted_index]
bt_easiness_corr = bt_easiness_corr.drop("BT Easiness")
bt_easiness_corr = bt_easiness_corr[abs(bt_easiness_corr) > 0.05]

import plotly.graph_objects as go

fig = go.Figure(
    data=go.Bar(
        x=bt_easiness_corr.index,
        y=bt_easiness_corr.values,
        marker=dict(color=bt_easiness_corr.values, colorscale="balance"),
        text=bt_easiness_corr.values.round(2),
        textposition="inside",
        cliponaxis=True,
    )
)

fig.update_traces(textangle=90)

fig.update_layout(
    title="Correlation of 'BT Easiness' with Word Origin Language",
    xaxis_title="Columns",
    yaxis_title="Correlation",
    xaxis_tickangle=-45,
    yaxis=dict(automargin=True),
    uniformtext_minsize=12,
    uniformtext_mode="show",
    height=700,
    width=1000,
)

fig.show()

In [10]:
origins_df = pd.DataFrame(df["BT Easiness"]).join(
    pd.DataFrame(df["pos_counts"].tolist()).fillna(0)
)

correlation_matrix = origins_df.corr()
bt_easiness_corr = correlation_matrix["BT Easiness"]
sorted_index = bt_easiness_corr.sort_values(ascending=False).index
bt_easiness_corr = bt_easiness_corr[sorted_index]
bt_easiness_corr = bt_easiness_corr.drop("BT Easiness")

import plotly.graph_objects as go

fig = go.Figure(
    data=go.Bar(
        x=bt_easiness_corr.index,
        y=bt_easiness_corr.values,
        marker=dict(color=bt_easiness_corr.values, colorscale="balance"),
        text=bt_easiness_corr.values.round(2),
        textposition="inside",
        cliponaxis=True,
    )
)

fig.update_traces(textangle=90)

fig.update_layout(
    title="Correlation of 'BT Easiness' with Parts of Speech",
    xaxis_title="Columns",
    yaxis_title="Correlation",
    xaxis_tickangle=-45,
    yaxis=dict(automargin=True),
    uniformtext_minsize=12,
    uniformtext_mode="show",
    height=700,
    width=1200,
)

fig.show()

In [24]:
origins_df = df[
    [
        "BT Easiness",
        "mean_syllable_count",
        "num_sentences",
        "mean_sentence_length",
        "mean_word_length",
        "num_words",
    ]
]

correlation_matrix = origins_df.corr()
bt_easiness_corr = correlation_matrix["BT Easiness"]
sorted_index = bt_easiness_corr.sort_values(ascending=False).index
bt_easiness_corr = bt_easiness_corr[sorted_index]
bt_easiness_corr = bt_easiness_corr.drop("BT Easiness")

import plotly.graph_objects as go

fig = go.Figure(
    data=go.Bar(
        x=bt_easiness_corr.index,
        y=bt_easiness_corr.values,
        marker=dict(color=bt_easiness_corr.values, colorscale="balance"),
        text=bt_easiness_corr.values.round(2),
        textposition="inside",
        cliponaxis=True,
    )
)

fig.update_traces(textangle=90)

fig.update_layout(
    title="Correlation of 'BT Easiness' with Sentence Composition Features",
    xaxis_title="Columns",
    yaxis_title="Correlation",
    xaxis_tickangle=-45,
    yaxis=dict(automargin=True),
    uniformtext_minsize=12,
    uniformtext_mode="show",
    height=700,
    width=700,
)

fig.show()

In [12]:
fig = px.histogram(df["mean_syllable_count"], nbins=30)
fig.update_layout(bargap=0.1)

fig.show()

In [18]:
fig = px.scatter(
    df,
    x=df["mean_syllable_count"],
    y=df["BT Easiness"],
    title="Scatterplot of Mean Syllable Count vs BT Easiness",
)
fig.show()

In [13]:
fig = px.histogram(df["num_sentences"], nbins=30)
fig.update_layout(bargap=0.1)

fig.show()

In [20]:
fig = px.scatter(
    df,
    x=df["num_sentences"],
    y=df["BT Easiness"],
    title="Scatterplot of Number of Sentences vs BT Easiness",
)
fig.show()

In [14]:
fig = px.histogram(df["mean_sentence_length"], nbins=30)
fig.update_layout(bargap=0.1)

fig.show()

In [21]:
fig = px.scatter(
    df,
    x=df["mean_sentence_length"],
    y=df["BT Easiness"],
    title="Scatterplot of Mean Sentence Length vs BT Easiness",
)
fig.show()

In [15]:
fig = px.histogram(df["mean_word_length"], nbins=30)
fig.update_layout(bargap=0.1)

fig.show()

In [22]:
fig = px.scatter(
    df,
    x=df["mean_word_length"],
    y=df["BT Easiness"],
    title="Scatterplot of Mean Word Length vs BT Easiness",
)
fig.show()

In [16]:
fig = px.histogram(df["num_words"], nbins=30)
fig.update_layout(bargap=0.1)

fig.show()

In [23]:
fig = px.scatter(
    df,
    x=df["num_words"],
    y=df["BT Easiness"],
    title="Scatterplot of Number of Words vs BT Easiness",
)
fig.show()

## Models

## Model Training

## Results and Analysis

## Conclusion

## References

http://etym.org/